In [1]:
import torch
import or_suite
import gym
from garage import wrap_experiment
from garage.envs import GymEnv, normalize
from garage.experiment.deterministic import set_seed
from garage.experiment import deterministic
from garage.sampler import LocalSampler
from garage.torch.algos import TRPO
from garage.torch.policies import GaussianMLPPolicy
from garage.torch.value_functions import GaussianMLPValueFunction
from garage.trainer import Trainer

2022-02-07 16:56:09.118820: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/gurobi912/linux64/lib
2022-02-07 16:56:09.118837: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Generic packages.  Expect some 'warnings' because you have no GPU

In [2]:
hyper_parameters = {
    'hidden_sizes': [32, 32], # neural network parameters
    'max_kl': 0.01, # "essentially" some function of the policy regularization
    'gae_lambda': 0.97, # no clue
    'discount': 0.99, # Oh, this will do infinite horizon with discount instead of finite horizon
    'n_epochs': 999, # num episodes
    'batch_size': 1024, # batches used for computing gradient steps
}

In [3]:
CONFIG =  or_suite.envs.env_configs.ambulance_metric_default_config
CONFIG['epLen'] = 500
env = gym.make('Ambulance-v0', config=CONFIG)
env._max_episode_length

500

In [15]:
@wrap_experiment
def trpo_ambulance(ctxt=None, seed=1):
    """Train TRPO with MultipleSuppliers-v0 environment.
    Args:
        ctxt (garage.experiment.ExperimentContext): The experiment
            configuration used by Trainer to create the snapshotter.
        seed (int): Used to seed the random number generator to produce
            determinism.
    """
    set_seed(seed)
    CONFIG =  or_suite.envs.env_configs.ambulance_metric_default_config
    CONFIG['epLen'] = 500
    CONFIG['num_ambulance'] = 25
    env = gym.make('Ambulance-v0', config=CONFIG)
    env = GymEnv(env, max_episode_length = CONFIG['epLen'])
    trainer = Trainer(ctxt)
    print(env.spec)
    policy = GaussianMLPPolicy(env.spec,
                               hidden_sizes=[32, 32],
                               hidden_nonlinearity=torch.tanh,
                               output_nonlinearity=None)

    value_function = GaussianMLPValueFunction(env_spec=env.spec,
                                              hidden_sizes=(32, 32),
                                              hidden_nonlinearity=torch.tanh,
                                              output_nonlinearity=None)
    sampler = LocalSampler(agents=policy,
                           envs=env,
                           max_episode_length=env.spec.max_episode_length) # TODO: Issue here

    algo = TRPO(env_spec=env.spec,
                policy=policy,
                value_function=value_function,
                sampler=sampler,
                discount=0.99,
                center_adv=False)

    trainer.setup(algo, env)
    trainer.train(n_epochs=100, batch_size=1024)

In [16]:
trpo_ambulance(seed = 1)

2022-02-07 17:00:46 | [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] Logging to /home/sean/Programming/ORSuite/examples/data/local/experiment/trpo_ambulance_17
2022-02-07 17:00:46 | [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] Logging to /home/sean/Programming/ORSuite/examples/data/local/experiment/trpo_ambulance_17
2022-02-07 17:00:46 | [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] Logging to /home/sean/Programming/ORSuite/examples/data/local/experiment/trpo_ambulance_17
2022-02-07 17:00:46 | [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] Logging to /home/sean/Programming/ORSuite/examples/data/local/experiment/trpo_ambulance_17
2022-02-07 17:00:46 | [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] [trpo_ambulance] Logging to /home

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1 and 25x32)